In [2]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 480 kB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 40.0 MB 28.2 MB/s eta 0:00:01
     |████████████████████████████████| 194 kB 16.9 MB/s eta 0:00:01
     |████████████████████████████████| 132 kB 19.5 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 20.7 MB/s eta 0:00:01
     |████████████████████████████████| 179 kB 20.8 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 23.4 MB/s eta 0:00:01
     |████████████████████████████████| 116 kB 15.5 MB/s eta 0:00:01
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.1
    Uninstalling tqdm-4.64.1:
      Successfully uninstalled tqdm-4.64.1
  Attempting uninstall: requests
    Found existing installation: requests 2.28.1
    Uninstalling requests-2.28.1:
      Successfully uninstalled requests-2.28.1
  Attempting uninstall: fsspec
    Found existing installation: fss

In [4]:
pip install --upgrade datasets pyarrow


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
import pandas as pd
from datasets import Dataset, DatasetDict

def format_messages_to_text(messages):
    """
    Convert messages to a specific text format for training
    Follows Llama 3 chat template conventions
    """
    formatted_text = ""
    for message in messages:
        if message['role'] == 'user':
            formatted_text += f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{message['content']}<|eot_id|>"
        elif message['role'] == 'assistant':
            formatted_text += f"<|start_header_id|>assistant<|end_header_id|>\n\n{message['content']}<|eot_id|>"
    formatted_text += "<|begin_of_text|>"
    return formatted_text

def prepare_dataset(json_path):
    # Load your data
    with open(json_path, "r") as f:
        data = json.load(f)

    # Convert to training format
    training_data = []
    for item in data:
        instruction = item.get("﻿instruction") or item.get("instruction")
        output = item["output"]
        
        # Skip if any field is missing
        if not instruction or not output:
            continue
        
        # Format for Llama-3 fine-tuning
        messages = [
            {"role": "user", "content": instruction},
            {"role": "assistant", "content": output}
        ]
        
        # Convert messages to formatted text
        text = format_messages_to_text(messages)
        training_data.append({"text": text})

    # Convert to Hugging Face dataset
    dataset = Dataset.from_list(training_data)

    # Split into train and validation (95% train, 5% validation)
    split_dataset = dataset.train_test_split(test_size=0.05)

    # Save dataset
    split_dataset.save_to_disk("processed_dataset")

    return split_dataset

# Run the preparation
dataset = prepare_dataset("/workspace/FINETUNING/dataset/ml_dataset.json")
print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Saving the dataset (1/1 shards): 100%|██████████| 473/473 [00:00<00:00, 57358.21 examples/s]

Train dataset size: 8981
Test dataset size: 473
